In [1]:
# Square Models and Long-Short Term Memory Netwirks
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [3]:
lstm = nn.LSTM(3, 3)  # input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a swquence of length 5

# initialize the huidden state
hidden = (torch.randn(1, 1, 3),
         torch.randn(1, 1, 3))
for i in inputs:
    # Step through the sequence one element as a time
    # after each epoch, hidden contains the hidden state
    out, hidden = lstm(i.view(1, 1, -1))

inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[ 0.2486, -0.0525, -0.2524]],

        [[ 0.1750, -0.0048, -0.1143]],

        [[-0.0102,  0.0536, -0.1400]],

        [[-0.0357,  0.0877, -0.0192]],

        [[ 0.2145,  0.0192, -0.0337]]], grad_fn=<CatBackward>)
(tensor([[[ 0.2145,  0.0192, -0.0337]]], grad_fn=<ViewBackward>), tensor([[[ 0.2984,  0.0952, -0.1647]]], grad_fn=<ViewBackward>))


In [22]:
# An LSTM for Part-of-Speech Tagging
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in  seq]
    return torch.tensor(idxs, dtype=torch.long)

training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [23]:
# Create the model
class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, target_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimentionally hidden_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, target_size)
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        return (torch.zeros(1, 1, self.hidden_dim),
               torch.zeros(1, 1, self.hidden_dim))
    
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
        embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores
        

In [26]:
# Train the model
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)


with torch.no_grad():
    inputs = prepate_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):
    for sentence, tags in training_data:
        model.zero_grad()
        # Clear out the hidden state of the LSTM
        model.hidden = model.init_hidden()
        # Get our inputs ready for the network, that is, turn them into Tensor of word indica\es.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        # Run out forward pass
        tag_scores = model(sentence_in)
        # Compute the loss, gradients, and update the parameters by calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)


tensor([[-1.0626, -1.0350, -1.2066],
        [-1.0886, -1.0228, -1.1916],
        [-1.0750, -1.0351, -1.1923],
        [-1.0498, -1.1141, -1.1338],
        [-1.0831, -1.0454, -1.1714]])
tensor([[-0.0442, -3.2796, -5.1827],
        [-4.9159, -0.0137, -5.0661],
        [-3.2334, -3.6538, -0.0675],
        [-0.0474, -3.7273, -3.8062],
        [-4.1249, -0.0228, -5.0619]])
